In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 40)
import numpy as np
import re
import spacy
import nltk
from nltk import FreqDist
import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Load  Data

In [5]:
data = pd.read_csv("comments_lemmentized.csv", index_col=0, sep=';')

In [6]:
data.shape

(1303, 1)

AttributeError: 'DataFrame' object has no attribute 'language'

In [8]:
data.head()

,lemmantized_reviews
0,"['premium', 'cottage', 'renovate', '..."
1,"['nice', 'environment', 'peaceful', ..."
2,"['expectation', 'high', 'read', 'rev..."
3,"['quite', 'center', 'parcs', 'france..."
4,"['new', 'year', 'trip', 'family', 's..."


In [9]:
#the lemmantized_reviews are interpreted as a string by the read_csv command. We convert it into a list. 
data['lemmantized_reviews'] = data['lemmantized_reviews'].str.replace(r"[\[\]\']", "").str.split(", ")

In [7]:
data['lemmantized_reviews'][0][:10] #checking if it's a list

['premium',
 'cottage',
 'renovate',
 'out',
 'ring',
 'park',
 'mean',
 'dome',
 'shop',
 'minute']

In [10]:
data.dropna(subset=['lemmantized_reviews'], inplace=True)

In [11]:
data.shape

(1303, 1)

## Remove words

In [12]:
# We list the words we want to get rid of
list_stop_words = ["good", "great", "one", "nt", "u2013", "", "would", "get", "center", "parcs"]

In [13]:
def remove_stop_word(x, list_stop_words=list_stop_words):
    clean_x = [l for l in x if l not in list_stop_words]
    return clean_x

In [14]:
data['lemmantized_reviews'] = data['lemmantized_reviews'].apply(remove_stop_word)

In [15]:
data.head()

,lemmantized_reviews
0,"[premium, cottage, renovate, ring, p..."
1,"[nice, environment, peaceful, facili..."
2,"[expectation, high, read, review, we..."
3,"[quite, france, normandy, old, franc..."
4,"[new, year, trip, family, stay, week..."


In [16]:
data.shape

(1303, 1)

## Building an LDA model

In [17]:
dictionary = corpora.Dictionary(data['lemmantized_reviews'])

In [18]:
len(dictionary)

10911

In [19]:
doc_term_matrix = [dictionary.doc2bow(rev) for rev in data['lemmantized_reviews']]

In [20]:
import warnings
warnings.filterwarnings('ignore')

# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
num_topics = 5
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, 
                num_topics=num_topics, 
                alpha=[0.0001] * num_topics, 
                eta=[0.0001] * len(dictionary),
                chunksize=2000,
                passes=4,
                random_state=100,
               )

In [21]:
lda_model.print_topics(num_words=8)

[(0,
  '0.013*"cottage" + 0.008*"time" + 0.007*"restaurant" + 0.007*"kid" + 0.006*"pool" + 0.006*"family" + 0.006*"day" + 0.006*"staff"'),
 (1,
  '0.008*"area" + 0.008*"park" + 0.007*"day" + 0.007*"lot" + 0.007*"activity" + 0.007*"pool" + 0.006*"site" + 0.006*"restaurant"'),
 (2,
  '0.014*"cottage" + 0.008*"pool" + 0.008*"area" + 0.007*"park" + 0.007*"staff" + 0.007*"time" + 0.006*"child" + 0.006*"restaurant"'),
 (3,
  '0.009*"time" + 0.008*"activity" + 0.008*"kid" + 0.008*"staff" + 0.007*"much" + 0.006*"park" + 0.006*"also" + 0.006*"english"'),
 (4,
  '0.013*"cottage" + 0.008*"time" + 0.007*"really" + 0.007*"kid" + 0.006*"activity" + 0.006*"pool" + 0.006*"park" + 0.006*"child"')]

## Topic visualization

In [22]:
# Visualize the topics
pd.options.display.max_colwidth = 2000
viz = pyLDAvis.gensim.prepare(lda_model, doc_term_matrix, dictionary, mds='tsne')

In [23]:
pyLDAvis.enable_notebook()
viz
#pyLDAvis.show(viz)

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      600.193481  184.534454       1        1  28.352026
2     -229.938095 -447.619110       2        1  22.315125
0     -546.712769   94.912498       3        1  17.386490
1      373.883514 -401.620880       4        1  16.443399
3       11.660259  334.095062       5        1  15.502956, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
644   Default   216.000000       speak   216.000000  30.0000  30.0000
608   Default   353.000000     english   353.000000  29.0000  29.0000
276   Default   360.000000      french   360.000000  28.0000  28.0000
7     Default  1261.000000     cottage  1261.000000  27.0000  27.0000
428   Default   356.000000       villa   356.000000  26.0000  26.0000
287   Default   448.000000        much   448.000000  25.0000  25.0000
224   Default   664.000000        area   664.000000  24.0000  24.0000
75    Default   144.000000        bite   144.000000  23.0000  23.0000
303   Default   652.000000       staff   652.000000  22.0000  22.0000
283   Default   137.000000       lodge   137.000000  21.0000  21.0000
243   Default   433.000000        site   433.000000  20.0000  20.0000
21    Default   194.000000          le   194.000000  19.0000  19.0000
274   Default   240.000000        find   240.000000  18.0000  18.0000
40    Default   667.000000  restaurant   667.000000  17.0000  17.0000
14    Default   185.000000          go   185.000000  16.0000  16.0000
106   Default   759.000000    activity   759.000000  15.0000  15.0000
307   Default   170.000000         try   170.000000  14.0000  14.0000
260   Default    74.000000        bois    74.000000  13.0000  13.0000
279   Default   111.000000        help   111.000000  12.0000  12.0000
172   Default   567.000000         lot   567.000000  11.0000  11.0000
33    Default   761.000000        park   761.000000  10.0000  10.0000
37    Default   325.000000       price   325.000000   9.0000   9.0000
48    Default   876.000000        time   876.000000   8.0000   8.0000
310   Default   240.000000       visit   240.000000   7.0000   7.0000
247   Default   526.000000        stay   526.000000   6.0000   6.0000
448   Default   243.000000         amp   243.000000   5.0000   5.0000
99    Default    83.000000        tire    83.000000   4.0000   4.0000
18    Default   747.000000         kid   747.000000   3.0000   3.0000
130   Default   673.000000         day   673.000000   2.0000   2.0000
609   Default   217.000000        euro   217.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
22     Topic5    98.263031      little   426.144318   0.3970  -5.2326
48     Topic5   165.184570        time   876.333557   0.1955  -4.7132
3406   Topic5    19.105303        wire    45.469498   0.9971  -6.8702
253    Topic5   115.804901        also   566.994263   0.2757  -5.0683
247    Topic5   108.515472        stay   526.771179   0.2843  -5.1333
18     Topic5   139.110382         kid   747.950439   0.1821  -4.8849
800    Topic5    42.182392        love   141.778702   0.6519  -6.0782
66     Topic5    99.605278        nice   487.612671   0.2758  -5.2190
428    Topic5    78.282127       villa   356.933105   0.3469  -5.4599
95     Topic5    37.966129        swim   125.382553   0.6695  -6.1835
163    Topic5    94.762756     holiday   510.993103   0.1792  -5.2688
152    Topic5    53.822727      france   220.789078   0.4526  -5.8345
510    Topic5    48.801361       think   191.507950   0.4970  -5.9325
33     Topic5   116.145187        park   761.808472  -0.0167  -5.0654
344    Topic5    73.773941        like   371.442078   0.2478  -5.5192
72     Topic5    86.429329        well   503.309326   0.1023  -5.3609
34     Topic5   112.795799        pool   796.516235  -0.0905  -5.0946
77     Topic5    85.669487       could   499.812775